<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%201/1_8_1_Kalifornien_Hauspreise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.8 Hauspreise in Kalifornien

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_housing.jpg">

## Was wir erreichen wollen

Ein Wohnungsgebiet in Kalifornien wird durch verschiedene Attribute beschrieben, z.B. durch die Anzahl der Häuser oder die Nähe zum Strand

Wir wollen in diesem Einstiegsprojekt ein Modell datengetrieben entwickeln, welches den Median-Preis eines Hauses einer beliebigen Gegend in Kalifornien berechnen kann.

Das könnte etwa in Form einer Funktion sein, wie folgt:

In [ ]:
def calculate_price(number_rooms, location, machine_learning_model):
    # number_rooms und location sind Eingaben in unser Modell
    # price ist die Ausgabe
    price = machine_learning_model.predict(number_rooms, location)
    return price

## Installieren relevanter Bibliotheken

Zunächst müssen wir einige Programmierbibliotheken importieren, welche wir verwenden wollen:

In [ ]:
import os # u.a. zur Entwicklung plattformübergreifender Systempfade
import yaml # für Konfigurationen
import pandas as pd # Datenmanagement
import numpy as np # Hilfsfunktionen für mathematische Operationen

# Datenvisualisierung
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split # Datensplits
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Modellevaluierung

## Anlegen von Dateipfaden

Es ist sinnvoll, Dateipfade zu Beginn des Projekts einmal in "Konstanten" (CAPSLOCK) Variablen zu schreiben:

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.getcwd()
FILE_PATH = "housing.csv"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

## Einlesen unserer Daten

Mit nun importierten Bibliotheken und angelegten Dateipfaden können wir unsere Daten mit `pandas` importieren:

In [ ]:
df = pd.read_csv(FILE_PATH) # Wir lesen die Datei housing.csv ein
df.head()

## Erste Dateiüberprüfung

Wir haben unsere Daten nun im Arbeitsspeicher und können einmal überprüfen, wie viele unserer Datenpunkte fehlende Werte enthalten:

In [ ]:
# wie viele Datenpunkte haben nicht-vollständige Attribute?
num_missing = len(df) - len(df.dropna())
ratio_missing = num_missing / len(df) 
# wir konvertieren in Prozente und runden auf 3 Nachkommastellen
ratio_missing = np.round(ratio_missing * 100, 3) 
print(f"{ratio_missing}% der Daten ({num_missing} Punkte) haben fehlende Werte!")

Schauen wir uns einmal die Daten an, die fehlende Werte haben:

In [ ]:
df.loc[df.isnull().sum(axis=1) == 1] # ein kleiner Trick zum anzeigen von Daten mit fehlenden Attributen

Es scheint so, als hätten wir in der Spalte `total_bedrooms` fehlende Werte. Ein weiterer Blick auf die Daten bestätigt uns das:

In [ ]:
df.isnull().sum(axis=0)

Wir könnten uns in einem Projekt gedanken machen, wie wir mit diesen fehlenden Werten umgehen. Hier machen wir es ganz einfach: Wir werden diese Daten aus unserem Datensatz einfach entfernen:

In [ ]:
df = df.dropna() # löscht alle Zeilen mit fehlenden Attributen
df = df.reset_index(drop=True) # zählt unsere Daten neu durch

### Unsere Zielvariable
Nun schauen wir uns an, was wir überhaupt prognostizieren wollen: Den Median-Hauswert einer Wohngegend. Wie sind unsere Daten hier verteilt?

In [ ]:
sns.displot(data=df, x="median_house_value", kde=True);

Wir merken, dass es deutlich abweichende Median-Preise in den Wohngegenden gibt und dass in den Daten scheinbar ein oberes Limit bei 500.000 USD liegt.

Wenn wir nun die Daten weiter analysieren wollen, sollten wir vorher den Split in Training/Test-Daten machen. Damit wir unsere Daten sauber splitten, werden wir einen Stratified Split machen (Zielvariablenverteilung bleibt erhalten).

Wir werfen einen Blick auf unsere statistischen, beschreibenden Kennzahlen:

In [ ]:
description = df.describe()
description

Wir interessieren uns besonders für die Quantile des `median_house_value`. Diese wollen wir nutzen, um unsere Daten zu kategorisieren und basierend auf diesen Kategorien einen Datensplit zu machen.

In [ ]:
bins = [0] + list(description["median_house_value"][
    ["25%", "50%", "75%"]
].astype(int)) + [np.inf]
print(bins) # 0 bis 119.500 = 0 - 25% Quantil, 119.500 bis 179.700 = 25 - 50% Quantil, ...

Wir können nun ein Hilfsattribut zum Splitten der Daten aufstellen, was Hauswerte in diskrete Kategorien einteilt:

In [ ]:
df["house_cat"] = pd.cut(
    df["median_house_value"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

Schauen wir uns die Verteilung auf unserem Datensatz an:

In [ ]:
sns.countplot(data=df, x="house_cat");

Wofür haben wir dies nun gemacht? Wir können jetzt wunderbar unsere Daten so splitten, dass Trainings- und Testdaten eine gleiche (oder zumindest möglichst ähnliche) Verteilung der Zielvariable `median_house_value` besitzen:

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in split.split(df, df["house_cat"]):
    df_train = df.loc[train_index]
    df_test = df.loc[test_index]

Schauen wir uns die Verteilung auf den Trainingsdaten an:

In [ ]:
sns.countplot(data=df_train, x="house_cat");

Und auf den Testdaten:

In [ ]:
sns.countplot(data=df_test, x="house_cat");

Nun können wir die Hilfsvariable einfach wieder aus unseren Daten entfernen, da wir unseren Datensplit vollbracht haben:

In [ ]:
df_train = df_train.drop("house_cat", axis=1)
df_test = df_test.drop("house_cat", axis=1)